# Task 1 Multiplier

## Problem statement

In this task, we wish to make a multiplier quantum circuit. For this, we design the input to be two positive integers to a function which will process a quantum algorithm that makes the multiplier (see Draper adder) and returns the result in an integer.

You cannot use any implementation already designed by the framework. If possible, consider printing out your quantum circuit.

### Example usage

A = multiplier(5,6)

print(A)

30

### Bonus

Use your proposal to design different inputs, and check the limitations of your simulator and framework, consider number of qubits, time of execution, the depth of the quantum circuit and number of the gates.


### References

* Addition on a Quantum Computer
https://arxiv.org/pdf/quant-ph/0008033.pdf 

* T-count Optimized Design of Quantum Integer Multiplication  
https://arxiv.org/pdf/1706.05113.pdf 

* Quantum arithmetic with the Quantum Fourier Transform 
https://arxiv.org/pdf/1411.5949.pdf

## Import statements

The following block is to contain all necessary import statements for this notebook. Run it before running any other code cell.

In [166]:
import tequila as tq
import numpy as np
from numpy import binary_repr as binrep
from numpy import ceil, log2, pi, floor
from typing import Union

## QFT Review

TODO: a review of what QFT does

Consider a system of $n$ qubits in the computational basis $\{\left| j \right\rangle: j\in [N]\}$ with $N=2^n$. Then, the QFT operation is a linear map such that, for any computational basis state $\left| x\right\rangle$, we have
$$QFT\left| x\right\rangle = \frac1{\sqrt{N}} \sum_{j\in [N]} e^{i\frac{2\pi x j}{N}} \left| j\right\rangle.$$
The QFT operation can be implemented explicitly in a quantum circuit using $\mathcal{O}(n^2)$ quantum gates (using only Hadamard, and controlled phase gates), as follows.

In [129]:
def qft(bits: Union[int, list[int]]) -> tq.QCircuit:
    """
    Returns a circuit implementing QFT on n qubits, where n = len(bits)
    
    The list of bits is ordered in decreasing priority of bits, i.e. with the MSB as bits[0]
    """
    circ = tq.QCircuit()
    if isinstance(bits, int):
        n, bits = bits, list(range(bits))
    else:
        n = len(bits)
    
    for i in range(n-1):
        circ += tq.gates.H(bits[i])
        for j in range(i+1, n):
            m = j - i
            phi = pi / (2 ** m)
            circ += tq.gates.Phase(target=bits[i], control=bits[j], angle=phi)
    circ += tq.gates.H(bits[-1])
    
    return circ


def qft1(n_bits: int) -> tq.QCircuit:
    """
    Less general implementation of QFT used for multiplier
    """
    circ = tq.QCircuit()
    for i in range(n_bits):
        circ += tq.gates.H(n_bits + i)
        for c in range(i + 1, n_bits):
            circ += tq.gates.Phase(target=n_bits+i, control=n_bits+c, angle=pi/(2**(c-i)))
    circ += tq.gates.H(2 * n_bits - 1)
    
    return circ

In [136]:
n=3
print(qft1(n))
tq.draw(qft1(n))

circuit: 
H(target=(3,))
Phase(target=(3,), control=(4,), parameter=1.5707963267948966)
Phase(target=(3,), control=(5,), parameter=0.7853981633974483)
H(target=(4,))
Phase(target=(4,), control=(5,), parameter=1.5707963267948966)
H(target=(5,))
H(target=(5,))

                    ┌──┐
0: ───H─────────S────T─────────────────────
                │    │
1: ───T─────────@────┼H────────S───────────
                     │         │
2: ───Z^(1/8)────────@─────T───@───H───H───
                    └──┘


''

## Aside: Phase gates in Tequila

I used the tq.gates.Phase(...) gate to implement the gate given by
$$ R_l = \begin{bmatrix} 1 & 0 \\ 0 & e^{2\pi i / 2^l} \end{bmatrix} $$

## QFT Adder

TODO: a review of implementing a Draper adder using QFT, using reference (3)

We use this adder to implement the multiplier given below.

In [291]:
def qft_ancilla(bits: Union[int, list[int]], anc: int) -> tq.QCircuit:
    """
    Returns a circuit implementing QFT on (n+1) qubits, using 1 ancillary qubit, viz. anc, where n = len(bits)
    
    Preconditions:
        - if isinstance(bits, list): anc not in bits
    """
    if isinstance(bits, int):
        n, bits = bits, list(range(bits))
    else:
        n = len(bits)
    
    bits[:0], circ = [anc], tq.QCircuit()
    for i in range(len(bits)):
        circ += tq.gates.H(target=bits[i])
        for k in range(i+1, len(bits)):
            circ += tq.gates.Phase(target=bits[i], control=bits[k], angle=pi/(2**(k-i)))
#     circ += tq.gates.H(target=bits[-1])
    return circ  


def adder(n1: int, n2: int, verbose: bool = False) -> tuple[tq.QCircuit, int]:
    """
    Returns a circuit implementing a QFT adder to add two integers $n_1$ and $n_2$, and the sum $n_1 + n_2$
    """
    circ = tq.QCircuit()
    
    # initialize variables
    bin1, bin2 = binrep(n1), binrep(n2)
    n1_bits, n2_bits = int(floor(log2(n1)+1)), int(floor(log2(n2)+1))
    bin1, bin2 = '0' * (max(n1_bits, n2_bits) - n1_bits) + bin1, '0' * (max(n1_bits, n2_bits) - n2_bits) + bin2
    n_bits, n1_bits, n2_bits = 2 * max(n1_bits, n2_bits), max(n1_bits, n2_bits), max(n1_bits, n2_bits)
    if verbose:
        print(bin1, bin2)
    
    # prepare n1 and n2 in comp basis (first n_bits qubits is n1, second n_bits qubits is n2)
    # bit encoding
    for i in range(n1_bits):
        circ += tq.gates.X(target=i)
        if bin1[i] == '0':
            circ += tq.gates.X(target=i)
    for i in range(n2_bits):
        circ += tq.gates.X(target=n1_bits+i)
        if bin2[i] == '0':
            circ += tq.gates.X(target=n1_bits+i)
    
    # prepare QFT(n2 + 0)
    circ += qft_ancilla(bits=list(range(n1_bits, n_bits)), anc=n_bits)
    
    # reverse ordering for controlled phase, as shown in reference
    mapping = {}
    for i in range((n2_bits) // 2):
        circ += tq.gates.SWAP(first=n1_bits+i, second=n_bits-i-1)
#         if (n1_bits + i != n_bits - i - 1):
#             mapping.update({n1_bits + i: n_bits - i - 1})
#     print(mapping)
#     circ = circ.map_qubits(mapping)
    
    # iterate through, and add controlled phase gates
    for i in range(n1_bits):
        circ += tq.gates.Phase(control=i, target=n1_bits, angle=pi/(2**(i+1)))
    for i in range(1, n2_bits + 1):
        for j in range(i-1, n1_bits):
            circ += tq.gates.Phase(control=j, target=n1_bits+i, angle=pi/(2**(j-i+1)))
    
    # undo the SWAP, to prepare for inverse QFT
    for i in range((n2_bits) // 2):
        circ += tq.gates.SWAP(first=n1_bits+i, second=n_bits-i-1)
    
    # inverse QFT
    circ += qft_ancilla(bits=list(range(n1_bits, n_bits)), anc=n_bits).dagger()
    
#     for i in range((n2_bits + 1) // 2):
#         circ += tq.gates.SWAP(first=n1_bits+i, second=n_bits-i)
    
    wfn = tq.simulate(circ)
    if verbose:
        print(wfn)
    val = wfn.keys()
    for k in val:
        val = k.integer
        break
    val -= n1 * 2**(n2_bits + 1)
    if verbose:
        print(n1 * 2**(n2_bits + 1), val)
    return (circ, val)

In [293]:
adder(3, 3, verbose=True)

11 11
+0.3536e^(-0.8125πi)|11100> +0.1913e^(+0.0625πi)|11010> +0.8536e^(-0.3125πi)|11101> +0.1913e^(+0.5625πi)|11011> +0.2706e^(-0.1875πi)|11111> 
24 4


(circuit: 
 X(target=(0,))
 X(target=(1,))
 X(target=(2,))
 X(target=(3,))
 H(target=(4,))
 Phase(target=(4,), control=(2,), parameter=1.5707963267948966)
 Phase(target=(4,), control=(3,), parameter=0.7853981633974483)
 H(target=(2,))
 Phase(target=(2,), control=(3,), parameter=1.5707963267948966)
 H(target=(3,))
 SWAP(target=(2, 3), control=())
 Phase(target=(2,), control=(0,), parameter=1.5707963267948966)
 Phase(target=(2,), control=(1,), parameter=0.7853981633974483)
 Phase(target=(3,), control=(0,), parameter=3.141592653589793)
 Phase(target=(3,), control=(1,), parameter=1.5707963267948966)
 Phase(target=(4,), control=(1,), parameter=3.141592653589793)
 SWAP(target=(2, 3), control=())
 H(target=(3,))
 Phase(target=(2,), control=(3,), parameter=-1.5707963267948966)
 H(target=(2,))
 Phase(target=(4,), control=(3,), parameter=-0.7853981633974483)
 Phase(target=(4,), control=(2,), parameter=-1.5707963267948966)
 H(target=(4,)),
 4)

## QFT Multiplier

An attempt at direct translation of the algorithm from reference (3)

In [296]:
def multiplier(num1: int, num2: int, verbose: bool = False) -> int:
    """
    Inputs:
        num1 : integer positive value that is the first parameter to the multiplier function,
        num2 : integer positive value that is the second parameter to the multiplier function.
    
    Preconditions:
        num1 > 0
        num2 > 0
    
    Output:
        the positive integer value of the multiplication between number_1 and number_2
     """

    # initialize variables
    bin1, bin2 = binrep(num1), binrep(num2)
    n1_bits, n2_bits = int(floor(log2(num1)+1)), int(floor(log2(num2)+1))
    bin1, bin2 = '0' * (max(n1_bits, n2_bits) - n1_bits) + bin1, '0' * (max(n1_bits, n2_bits) - n2_bits) + bin2
    n_bits, n1_bits, n2_bits = 2 * max(n1_bits, n2_bits), max(n1_bits, n2_bits), max(n1_bits, n2_bits)
    print(bin1, bin2)
    
    circ = tq.QCircuit()    # initialize circuit
    
    # bit encoding
    for i in range(n1_bits):
        circ += tq.gates.X(target=i)
        print(i)
        if bin1[i] == '0':
            circ += tq.gates.X(target=i)
    for i in range(n2_bits):
        circ += tq.gates.X(target=n1_bits+i)
        if bin2[i] == '0':
            circ += tq.gates.X(target=n1_bits+i)
    
    tq.draw(circ, backend='qiskit')
    
    # QFT on output register    
    mapping = {j : n_bits + j for j in range(n_bits)}
    circ += qft(n_bits).map_qubits(mapping)
#     tq.draw(circ)
    
    # Controlled rotations
    for i in range(n1_bits):  # iterate over bits of bin1
        for k in range(n2_bits):  # iterate over bits of bin2
            circ += tq.gates.Rz(control=[n1_bits + i + k, n1_bits - i - 1], target=n_bits+i, angle=pi/2**(k+1))
            
        for k in range(n1_bits): # iterate over bits of bin1
            for m in range(n1_bits - k): # iterate over remaining bits
                circ += tq.gates.Rz(angle=pi/(2**m), control=[n1_bits + k + m, n1_bits - i - 1], target=2*n1_bits+i+k+1)
    
    # Inverse QFT on output register    
    circ += qft(n_bits).map_qubits(mapping).dagger()
    
    wfn = tq.simulate(circ)
#     print(wfn.items())
    extra = num1 * ( 2**(n_bits + n2_bits) ) + num2 * ( 2**(n_bits) )
    m, n = 0, None
    
    # Ideally, this would give back a single computational basis state, 
    # but my code has a bug, and we get back multiple comp basis states...
    for k in wfn.keys():
        print(k.integer - extra)
        if np.linalg.norm(wfn.state[k]) > m:
            n, m = k.integer - extra, np.linalg.norm(wfn.state[k])
    print(n)
    
    return n # the result of the quantum circuit into an integer value

In [297]:
multiplier(4,8)

0100 1000
0
1
2
3
16
144
80
208
48
176
112
240
16


16

In [202]:
print(3 * (2**(3+5)) + 6 * (2**5))
l = [1,2,3]
l[:0] = [0]
print(l)

960
[0, 1, 2, 3]


## Discussion

I was not able to complete the task properly, partly because I could not understand where the bug in my code is. I believe that I translated over everything from the reference used into code, so I am comfortable with the algorithm (and, after several sleepless nights of bug-digging, I think I know the algorithm well enough to recite it while half-asleep). I believe the fault lies in some way in which I misinterpreted how Tequila implements its gates.

Nevertheless, here I will try giving an asymptotic analysis of my code's running time and memory usage.

First, we use $2n$ bits to encode the numbers $n_1$ and $n_2$ using a binary bit encoding, where we explicitly define $n=\max\{\lfloor\log(n_1)+1\rfloor,\lfloor\log(n_2)+1\rfloor\}$ to avoid any confusion between quasi-polynomial and polynomial scaling. Next, the QFT and inverse QFT at the beginning and end of the multiplication routine require $\mathcal{O}(n^2)$ elementary operations. It is straightforward to observe that the controlled phase rotations in between the QFT subroutines take an additional $\mathcal{O}(n^3)$ elementary operations. So, overall, this implementation requires $\mathcal{O}(n^3)$ operations. We used $\mathcal{O}(n)$ ancillary qubits, and the depth of circuit was $\mathcal{O}(n^3)$.

The way in which Tequila parsed $R_Z$ and Phase gates was tedious because it compiles them into $R_X$ and $H$ gates which made it difficult to debug. I tried translating everything into QFT as a last-minute decision, and have included what I managed to write in a separate Python file.

As an alternative implementation, we could use a tradeoff between time and space by using a unary encoding, or do something very naive and use gray-code encoding and repetitive addition with decrementing one register after each successive addition.